In [0]:
import json
import pandas as pd

with open("meta_Musical_Instruments.json") as f:
    meatadata_json_data = [json.loads(line) for line in f]

In [0]:
df_metadata = pd.DataFrame(meatadata_json_data)
df_metadata = df_metadata[['asin','description']]

In [0]:
import re
regEx = re.compile('[^a-z]+')
def cleanReviews(reviewText):
    if isinstance(reviewText, list):
      reviewText = ''.join(reviewText)
    reviewText = reviewText.lower()
    reviewText = regEx.sub(' ', reviewText).strip()
    return reviewText

In [0]:
df_metadata = df_metadata.drop_duplicates("asin", keep="last")


In [13]:
df_metadata = df_metadata.dropna(subset=['description'])
df_metadata.count()

asin           102479
description    102479
dtype: int64

In [0]:
df_metadata["description_clean"] = df_metadata["description"].apply(cleanReviews)
del df_metadata["description"]

In [23]:
df_metadata.reset_index(inplace=True, drop=False)
df_metadata

,index,asin,description_clean
0,0,0000989983,cricket rubbing the spine with the wooden stic...
1,1,0000098906,frog rubbing its spine with the wooden stick u...
2,2,0041291905,vivaldi s famous set of four violin concertos ...
3,3,0060015500,the turn of the screw op vocal score published...
4,5,0470536454,an updated classic covering the latest techniq...
...,...,...,...
102474,120395,B01HJDOF2Y,this packages includes pcs pdt true bypass swi...
102475,120396,B01HJDJ1PA,this packages includes pcs pdt true bypass swi...
102476,120397,B01HJEFFTK,these high quality clear crystal singing bowls...
102477,120398,B01HJEHEH6,frog rubbing its spine with the wooden stick u...


In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

description = df_metadata["description_clean"]
countVector = CountVectorizer(max_features = 1000, stop_words='english') 
transformedDescription = countVector.fit_transform(description) 

dfDescription = pd.DataFrame(transformedDescription.A, columns=countVector.get_feature_names())

In [0]:
import numpy as np
X = np.array(dfDescription)
 # create train and test
tpercent = 0.9
tsize = int(np.floor(tpercent * len(dfDescription)))
dfDescription_train = X[:tsize]
dfDescription_test = X[tsize:]
#len of train and test
lentrain = len(dfDescription_train)
lentest = len(dfDescription_test)

In [0]:
from sklearn.neighbors import NearestNeighbors
neighbor = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(dfDescription_train)

In [27]:
i = 4
a = neighbor.kneighbors([dfDescription_test[i]])
related_product_list = a[1]
for item in related_product_list:
  # if df_metadatat
  print(df_metadata["asin"][item])

81464    B00QHTR268
80495    B00PADLU24
81463    B00QHSQKZI
80600    B00PECQR7Y
80416    B00P7E5UZE
80418    B00P7EH2IM
84174    B00TY640E8
92196    B015C8DFIS
92198    B015C8JOGU
92187    B015BQ97OW
Name: asin, dtype: object
